In [ ]:
import pandas as pd
import numpy as np

import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
from rpy2.robjects.packages import importr
base = importr('base')
utils = importr('utils')

lme4 = importr('lme4')

# Example of toy rpy2 dataframe usage

In [ ]:
# Example of making R df
r_df = ro.DataFrame({'int_values': ro.IntVector([1,2,3]),
                     'str_values': ro.StrVector(['abc', 'def', 'ghi'])})

# Example automatically converting pd dataframe into R and calling glmer from python
# df = pd.DataFrame(np.random.rand(5,4), columns=['a','b','c','d'])
# with localconverter(ro.default_converter + pandas2ri.converter):
#     lme4.glmer('a ~ b + c + (1 | d)', data=df, family='binomial')


# Example from UCLA stats
https://stats.idre.ucla.edu/r/dae/mixed-effects-logistic-regression/


Another example of glmer:
https://cran.r-project.org/web/packages/ggeffects/vignettes/practical_logisticmixedmodel.html

In [ ]:
rdf = ro.r('hdp <- read.csv("https://stats.idre.ucla.edu/stat/data/hdp.csv")')
ro.r['hdp']

In [ ]:
ro.r('''hdp <- within(hdp, {
  Married <- factor(Married, levels = 0:1, labels = c("no", "yes"))
  DID <- factor(DID)
  HID <- factor(HID)
  CancerStage <- factor(CancerStage)
})''')

In [ ]:
# ro.r('''m <- glmer(remission ~ IL6 + CRP + CancerStage + LengthofStay + Experience +
#     (1 | DID), data = hdp, family = binomial, control = glmerControl(optimizer = "bobyqa"),
#     nAGQ = 10)''')

ro.r('''m <- glmer(remission ~ IL6 + CRP + CancerStage + LengthofStay + Experience +
    (1 | DID), data = hdp, family = binomial, 
    nAGQ = 10)''')

# print summary formatted
print(ro.r('summary(m)'))

# Alternative way to print same output: 
with localconverter(ro.default_converter + pandas2ri.converter):
    m_from_r = ro.conversion.rpy2py(ro.r['m'])
print(base.summary(m_from_r))

In [ ]:
# call glmer from python on converted data

with localconverter(ro.default_converter + pandas2ri.converter):
    pd_from_r_df = ro.conversion.rpy2py(ro.r['hdp'])
    
    
with localconverter(ro.default_converter + pandas2ri.converter):
    res = lme4.glmer('''remission ~ IL6 + CRP + CancerStage + LengthofStay + Experience +
    (1 | DID)''', 
               data=pd_from_r_df, 
               family='binomial', 
#                control='glmerControl(optimizer = "bobyqa")',
               nAGQ = 10)
    
# Prints the output, but includes printing all the data (because it's not known by a df name in R?)
print(base.summary(res))